# Building a Pandas data frame from Matlab experiment files in the Nauhaus lab.

This code builds a huge table, where each row is a different experiment, and each column is an experimental variable. The information is extracted from the .analyzer files, typical contained in the folder 'AnalyzerFiles'. A given 'AnalyzerFiles' folder will have experimental information associated with potentially every experiment ever performed in the Nauhaus Lab. At a minimum, the information related to each experiment is contained in the GUIs on the stimulus controller: e.g. animal name, screen distance, visual stimulus looping parameters, parameter lists, display type, recording method, etc. 

The table that gets built is in the form of a Pandas data frame. Pandas is an analysis tool built on Python. This allows for a ton of flexibility for filtering out different experiments. Examples are given at the end.


In [159]:
import os
import numpy as np
import pandas as pd
from scipy import io
from organizemat import organizemat #Local. Taken from S.O.
from mat2list import mat2list #My function. Interprets MATLAB vector into a Python list.

In [160]:
#type in the path location of this notebook
wdir = '/Users/in2293/Desktop/nlab_experiment_finder' #working directory
os.chdir(wdir)

#Analyzer files may be somewhere else on th eocmputer...
fileloc = '/Users/in2293/Desktop/Desktop/AnalyzerFiles' 

subfolders = [f.path for f in os.scandir(fileloc) if f.is_dir() ] #list of all the folder names
print('no animals = ', len(subfolders))  #Each folder is a separate animal.

pd.set_option("display.max_columns", None) #Show all columns of the dataframe

no animals =  605


# Build or load the data frame

# Build it...
The cell below takes about ~15 min to run. It loads every analyzer file and accumulates to the dataframe.

In [161]:

#Initialize the 3 data frames. We will append 1 row of data after loading each experiment
ACQcolumns = ['FPS', 'bin', 'timecourseBit', 'btwTrialShutter', 'MechbtwTrialShutter',
                'ROIcrop', 'sensorGain','Gamma', 'camera', 'chipSIZE']
Mdf = pd.DataFrame()
Pdf = pd.DataFrame()
Ldf = pd.DataFrame()
ACQdf = pd.DataFrame(columns = ACQcolumns)


unloaded_experiments = []

exp_count = 0;


for fi,f in enumerate(subfolders):  #loop each animal
    
      print(f)
        
      for anafile in os.scandir(f):  #loop each experiment from the given animal
        try:
            
            data = io.loadmat(anafile.path, struct_as_record=False, squeeze_me=True)
            
            #Other things I tried for loading MATLAB .mat file and the reasons they didn't work:
            #Analyzer = loadmat(anafile.path)  #Could use this instead of the above, but it takes too long.          
            #Analyzer = io.loadmat(anafile.path,simplify_cells = True) #Throws error for files containing video object
            
        except:  #Sometimes there are irrelevant files saved in each folder. 
                #I want to make sure they are irrelevant by storing their name.
            
            print('error loading ', anafile.path)    
            unloaded_experiments.append(anafile.path) #I want to know what files were not loaded
            continue
                
        #Remove unnecessary "syncInfo". 
        #Otherwise it takes forever to parse into a dict below.
        data_copy = data.copy()
        for k,v in data.items():
            if k[0:4] == 'sync':        
                data_copy.pop(k)

        #Organize loaded .mat file into something readable.  Ref: StackOverflow.
        Analyzer = organizemat(data_copy)   
        
        #Move header into the M dict
        header = Analyzer['__header__']
        try: #I found some analyzer files that were missing 'Analyzer', but had 'f1m' wtf???
            Analyzer = Analyzer['Analyzer']
        except:
            continue
        Analyzer['M']['header'] = header   
        
        #Experimental parameters are in 3 GUI windows at stimulus-controller:
        M = Analyzer['M'].copy() #Parameters in the "MW" GUI
        L = Analyzer['L'].copy() #Parameters in the "Looper" GUI
        P = Analyzer['P'].copy() #Parameters in the "paramList" GUI
        
        #Reformat ACQ
        if 'ACQ' in Analyzer: #Only widefield experiments have ACQ.  And some really old WF do not either.
            ACQ = dict(zip(ACQcolumns, [None]*len(ACQcolumns)))
            for i in ACQcolumns:
                if i in Analyzer['ACQ']:
                    ACQ[i] = Analyzer['ACQ'][i]
                
            for k,v in ACQ.items():
                if type(v) != float and type(v) != int:
                    ACQ[k] = str(v)

        #Reformat L
        if type(L['param'][0]) is str: #Asks if there is only one looping parameter
            L['param'] = [L['param']]  #Embed it to make it consistent with N>1 parameter case         
        n_loop_param = len(L['param'])  #number of looping parameters
        #mat2list converts a matlab vector creation (e.g. '0:45:315', or [0 45]), into a complete Python list.
        #Expanding the vector into a Python list allows for easier querying of the data frame.
        for i in range(n_loop_param):
            L['paramSymbol' + str(i+1)] = L['param'][i][0]
            L['paramValues' + str(i+1)] = str(mat2list(L['param'][i][1]))
            
            L['paramValuesMatlabStr' + str(i+1)] = L['param'][i][1]
        
        L.pop('param') #No longer needed. Redundant.
        
        #json_normalize turns it into a df, and helps to unpack some fields
        M = pd.json_normalize(M)
        L = pd.json_normalize(L)
        P = pd.json_normalize(P)
        if 'ACQ' in Analyzer:
            ACQ = pd.json_normalize(ACQ)
        
        #Reformat P data frame:
        #Create a data frame with one row: columns are the parameter symbols, entries are the values.
        columns = list(pd.DataFrame(P.param[0]).iloc[:,0]) #columns
        values = list(pd.DataFrame(P.param[0]).iloc[:,2]) #values
        columns = ['module'] + columns  #Add a new first element to the list
        values = [P.type[0]] + values  #append front with module: e.g. 'PG' for periodic grater
        P = pd.DataFrame(columns = columns)
        P.loc[0] = values 
     
        Mdf = Mdf.append(M,ignore_index = True)
        Ldf = Ldf.append(L,ignore_index = True)
        Pdf = Pdf.append(P,ignore_index = True)
        ACQdf = ACQdf.append(ACQ,ignore_index = True)
        
        exp_count += 1
        

Mdf.drop(columns = ['camera'],axis = 1,inplace = True) #This field is usually wrong, and redundant with ACQ.camera.        


/Users/in2293/Desktop/Desktop/AnalyzerFiles/sp2
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nr4
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nr4/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rf6
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rf6/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/wy5
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/wy5/grabs
/Users/in2293/Desktop/Desktop/AnalyzerFiles/ri2
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/ri2/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/dr4
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/dr4/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/wz3
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rj4
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rj4/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nq5
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nq5/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/xi5
/

error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rx2/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/wh5
/Users/in2293/Desktop/Desktop/AnalyzerFiles/mz9
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rw6
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rw6/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/xx5
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx5/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/ya2
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sn8
/Users/in2293/Desktop/Desktop/AnalyzerFiles/mz0
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sb3
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/sb3/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rt0
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rt0/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sm7
/Users/in2293/Desktop/Desktop/AnalyzerFiles/xw6
/Users/in2293/Desktop/Desktop/AnalyzerFiles/mz7
error loading  /Users/in2293/Desktop/Desktop/Anal

error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/ry0/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/wi7
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nm5
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nm5/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nb1
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nb1/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nh1
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nh1/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/ng5
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/ng5/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/si8
/Users/in2293/Desktop/Desktop/AnalyzerFiles/se3
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/se3/.DS_Store
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/se3/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sj7
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rp8
error loading  /Users/in2293/Desktop/Desktop/

error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rn5/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sw2
/Users/in2293/Desktop/Desktop/AnalyzerFiles/ra1
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/ra1/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nz0
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nz0/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/ra6
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/ra6/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sx1
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sw5
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rn2
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rn2/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nu3
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nu3/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rb9
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rb9/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles

/Users/in2293/Desktop/Desktop/AnalyzerFiles/wz7
/Users/in2293/Desktop/Desktop/AnalyzerFiles/dr0
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/dr0/.DS_Store
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/dr0/rs8
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/dr0/rs6
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/dr0/rt2
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/dr0/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sp8
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nr9
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nr9/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rj7
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rj7/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nq6
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nq6/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nx1
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nx1/desktop.ini
/Users/in2293/D

...

error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u004_168.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u006_139.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u004_169.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u002_001.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u000_051.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u027_022.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u000_237.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u002_000.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u000_236.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u000_050.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u028_009.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx

error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u004_011.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u006_041.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u000_129.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u022_002.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u022_003.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u000_128.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u029_018.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u029_019.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u026_033.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u026_032.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u004_119.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx

error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u028_013.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u028_012.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u012_075.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u035_006.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u010_025.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u035_007.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u010_024.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u006_122.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u004_314.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u004_172.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx0_u011_044.analyzer
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx0/xx

error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/sa0/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rx7
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rx7/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/xx9
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/xx9/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rw3
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rw3/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sn4
/Users/in2293/Desktop/Desktop/AnalyzerFiles/my4
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nl5
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nl5/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sn3
/Users/in2293/Desktop/Desktop/AnalyzerFiles/wh7
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rw4
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rw4/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rx0
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rx0/desktop

/Users/in2293/Desktop/Desktop/AnalyzerFiles/nh3
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/nh3/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rs2
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rs2/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sj5
/Users/in2293/Desktop/Desktop/AnalyzerFiles/se1
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rj72
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rj72/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/wo9
/Users/in2293/Desktop/Desktop/AnalyzerFiles/se6
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/se6/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/sj2
/Users/in2293/Desktop/Desktop/AnalyzerFiles/rs5
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/rs5/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/ry5
error loading  /Users/in2293/Desktop/Desktop/AnalyzerFiles/ry5/desktop.ini
/Users/in2293/Desktop/Desktop/AnalyzerFiles/nm0
error loading  /User

# Clean up and save the table as a .csv

In [163]:
print(exp_count)
print(Mdf.shape, Ldf.shape, Pdf.shape, ACQdf.shape)
print(exp_count)

#This requires something different for queries, which I have not yet figured out:
#df = pd.concat([Mdf,Ldf,Pdf,ACQdf],keys=['M', 'L', 'P', 'ACQ'],axis = 1)    #Create one single data frame with all the parameters.

df = pd.concat([Mdf,Ldf,Pdf,ACQdf],axis = 1)    #Create one single data frame with all the parameters.
df = df.drop_duplicates(subset=['anim', 'expt','WF']) #Remove redundant rows: copies of a file often exist.

df.to_csv('all_experiments') #saves to working dir

print(f'saved table with {df.shape[0]} experiments to ', wdir)

saved table with 7226 experiments to  /Users/in2293/Desktop/nlab_experiment_finder


# Load saved table

This is an alternative to the 'load' and 'save cells above.  You don't need the .analyzer files for this, only the saved .csv file.



In [170]:
df = pd.read_csv('all_experiments')

/Users/in2293/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,29,68) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Example: Create a table that only contains widefield Kalatsky retinotopy experiments.

Kalatsky experiments have some unique features. 
1) The looper only has ori = [0 90 180 270] \
2) The temporal period is really long.  e.g. > 800 frames. \
The above should narrow it down, but I include several other dependencies as well, just to make sure.



In [171]:
#Find all experiments where orientation was the only looping variable

#Make sure ori is the only looping parameter
Kdf = df.query("paramSymbol1 == 'ori'"). \
            query('paramSymbol2 != paramSymbol2'). \
            query('paramSymbol2 != paramSymbol3'). \
            query('paramSymbol3 != paramSymbol4') 
            

#ori loops through for cardinal directions: 
Kdf = Kdf.query("paramValues1 == '[0.0, 90.0, 180.0, 270.0]'")
    
print(f'n = {Kdf.shape[0]} experiments where ori is only looping variable: [0 90 180 270]')

#The bar is changing slowly
Kdf = Kdf.query('t_period > 800')
print(f'n = {Kdf.shape[0]}')

#The bar is drifting
Kdf = Kdf.query('separable == 0')
print(f'n = {Kdf.shape[0]}')

#The bar drifts over a large part of the screen
Kdf = Kdf.query('x_size > 100').query('y_size > 100')
print(f'n = {Kdf.shape[0]}')
      
#There is only one bar on the screen
Kdf = Kdf.query('s_freq < 1/80')
print(f'n = {Kdf.shape[0]}')

#Its a narrow bar and not a sinewave
Kdf = Kdf.query("st_profile == 'square'").query("s_duty < 0.4")
print(f'n = {Kdf.shape[0]} Kalatsky experiments')


Kdf = Kdf.query("WF == 1")  #Change this to 'twoP' if you want two-photon Kalatsky
print(f'n = {Kdf.shape[0]} widefield Kalatsky experiments')

#Kdf = Kdf.drop_duplicates(subset = ['anim'])
#print(f'n = {Kdf.shape[0]} animals in which widefield Kalatsky was run')


n = 2148 experiments where ori is only looping variable: [0 90 180 270]
n = 1716
n = 1716
n = 1594
n = 1591
n = 1591 Kalatsky experiments
n = 1309 widefield Kalatsky experiments


# Save table of Kalatsky experiments

In [167]:
Kdf.to_csv('Kalatsky table') #saves to working dir

print(f'saved table with {Kdf.shape[0]} experiments to ', wdir)


saved table with 1309 experiments to  /Users/in2293/Desktop/nlab_experiment_finder
